In [3]:
%reload_ext autoreload
%autoreload 2
os.chdir("..")
import toml
from core.dl_framework.utils import read_config

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%%bash 
pwd

/content/Template_Project


In [61]:
config_file = toml.load("configs/default_train_config.toml")
setup_config = read_config(config_file)

In [62]:
from core.dl_framework.learner import Learner
from core.dl_framework.callbacks import Recorder, Monitor
from core.dl_framework.data import get_dataset, Dataset, DataLoader, split_data
bs = setup_config["h_batch_size"]
x_train, y_train, x_test, y_test = get_dataset(setup_config["s_source"], setup_config["s_set"], setup_config["p_tmp_data_path"])
train_ds, test_ds =  Dataset(x_train, y_train), Dataset(x_test, y_test)
train_dl, valid_dl = DataLoader(train_ds, bs), DataLoader(test_ds, bs)

In [63]:
learn = Learner(train_ds, setup_config)
learn.fit(100)
# a = iter(learn.data.train_dl)
# b, c = next(a)
# learn.learn.model(learn.learn.data.train_ds.x.shape[1], 10)
# next(learn.learn.model.parameters()).device
learn.device
learn.cbh.Monitor.history

100%|██████████| 187/187 [00:00<00:00, 2294.13it/s]


epoch: 1/100	[valid_acc: 0.5205	valid_loss: 2.0435	train_loss: 2.1853]


100%|██████████| 187/187 [00:00<00:00, 2113.43it/s]


epoch: 2/100	[valid_acc: 0.6867	valid_loss: 1.7431	train_loss: 1.8848]


100%|██████████| 187/187 [00:00<00:00, 2189.92it/s]


...

epoch: 90/100	[valid_acc: 0.9248	valid_loss: 0.2662	train_loss: 0.2489]


100%|██████████| 187/187 [00:00<00:00, 2293.04it/s]


epoch: 91/100	[valid_acc: 0.9249	valid_loss: 0.2654	train_loss: 0.2479]


 97%|█████████▋| 724/750 [00:01<00:00, 654.22it/s]


KeyboardInterrupt: 

In [25]:
import numpy as np
test_dict = {"test": [1, 2,4, 4, 9, 1, 3, 4]}
max(test_dict["test"])

9

In [48]:
np.greater(5, 3)

True

In [29]:
%%timeit
test_num.max()

1.57 µs ± 284 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [41]:
%%timeit
max(test_dict["test"])


281 ns ± 10.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [47]:
import types
class test_class():
    def __init__(self, a):
        self.attr = a
    
    def attr(self):
        pass

test = test_class(1)

getattr(test_class, "attr")

<function __main__.test_class.attr(self)>

In [35]:
%%timeit
test1.sum()

2.09 µs ± 52.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [32]:
import numpy as np

In [16]:
class test1:
    def __init__(self, a):
        self.a = a
    def test_func(self):
        return self.a
a = test1(1)
setattr(a, "s", 123)
vars(a)

{'a': 1, 's': 123}

In [17]:
monitor = Monitor_Cb(["val_acc"])
early = EarlyStopping_Cb(monitor="val_loss")
recorder = Recorder_Cb("hi")
cbh = CallbackHandler([monitor, early, recorder])
cbh.Monitor_Cb.learn


NameError: name 'Monitor_Cb' is not defined

In [9]:
a = ["a"]
for i in a[:4]:
    print(i)

a
